In [274]:
import pandas as pd

#### Importando os Dados

In [275]:
df_final_data_sumulas = pd.read_excel("data/final_data_original.xlsx", sheet_name="Sumulas", index_col=None)
df_final_data_atletas = pd.read_excel("atletas.xlsx", index_col=None)
df_final_data_competicoes = pd.read_excel("data/final_data_original.xlsx", sheet_name="Campeonatos", index_col=None)
df_sumulas_novas = pd.read_csv("sumulas.csv")

In [276]:
df_final_data_sumulas.shape

(136618, 14)

#### Tirando Coluna Problemática

In [277]:
df_final_data_sumulas.drop(df_final_data_sumulas.columns[0], axis=1, inplace=True)
df_final_data_atletas.drop(df_final_data_atletas.columns[0], axis=1, inplace=True)
df_final_data_competicoes.drop(df_final_data_competicoes.columns[0], axis=1, inplace=True)

#### Verificando Colunas

In [278]:
df_final_data_sumulas.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição'],
      dtype='object')

In [279]:
df_final_data_atletas.columns

Index(['Nome Competidor', 'ID Competidor'], dtype='object')

In [280]:
df_final_data_competicoes.columns

Index(['ID Competição', 'Nome Competição'], dtype='object')

In [281]:
df_sumulas_novas.columns

Index(['Identificador', 'Nome Competição', 'Regata', 'Nome Competidor',
       'Pontuação Regata (Nha)', 'Classe Vela', 'Flag Name', 'Posição Geral',
       'Pontuação Regata', 'Descarte', 'Punição', 'Flotilha', 'Total', 'Net'],
      dtype='object')

#### Competições

In [282]:
df_final_data_competicoes = df_final_data_competicoes.sort_values(by="ID Competição").reset_index(drop=True)
df_final_data_competicoes.head(10)

,ID Competição,Nome Competição
0,1,Central and South American Championship 2021
1,2,Central and South American Championship 2022
2,3,Central and South American Championship 2023
3,4,European Championship 2021
4,5,European Championship 2022
5,6,European Championship 2023
6,7,Semaine Olympique Francaise De Voile 2022
7,8,Semaine Olympique Francaise De Voile 2023
8,9,Trofeo S.A.R Princesa Sofia 2022
9,10,Trofeo S.A.R Princesa Sofia 2023


In [283]:
df_final_data_atletas = df_final_data_atletas.sort_values(by="ID Competidor").reset_index(drop=True)
df_final_data_atletas.head(10)

,Nome Competidor,ID Competidor
0,"WILLIAM JONES, EVAN DEPAUL",1
1,"DOMINIK BUKSAK, SZYMON WIERZBICKI",2
2,"MARTINE GRAEL, KAHENA KUNZE",3
3,"RUGGERO TITA, CATERINA BANTI",4
4,"BLAIR TUKE, PETER BURLING",5
5,"IDA MARIE BAAD NIELSEN, MARIE THUSGAARD OLSEN",6
6,"HANNA KLINGA, LISA ORN",7
7,"DIEGO BOTIN, FLORIAN TRITTEL PAUL",8
8,"ANNA BURNET, JOHN GIMSON",9
9,"JORDI XAMMAR HERNANDEZ, NORA BRUGMAN CABOT",10


#### Fornecendo ID

In [284]:
def retorna_ID_competidor(nome_atleta:str):
    """
    Retorna o ID do competidor com base no nome do atleta.

    Parâmetros:
    nome_atleta (str): O nome do atleta cujo ID do competidor será retornado.

    Retorna:
    str: O ID do competidor correspondente ao nome do atleta especificado.

    Exemplo:
    >>> retorna_ID_competidor("DOMINIK BUKSAK, SZYMON WIERZBICKI")
    'ID correspondente'
    """
    return df_final_data_atletas.loc[df_final_data_atletas['Nome Competidor'] == nome_atleta, 'ID Competidor'].iloc[0]

In [285]:
def retorna_ID_competicao(nome_competicao: str):
    """
    Retorna o ID da competição com base no nome da competição.

    Parâmetros:
    nome_competicao (str): O nome da competição cujo ID será retornado.

    Retorna:
    str: O ID da competição correspondente ao nome da competição especificada.

    Exemplo:
    >>> retorna_ID_competicao("Nome da Competição")
    'ID correspondente'
    """
    return df_final_data_competicoes.loc[df_final_data_competicoes['Nome Competição'] == nome_competicao, 'ID Competição'].iloc[0]


#### Adicionando ID nos Novos Dados

In [286]:
df_sumulas_novas["ID Competidor"] = df_sumulas_novas["Nome Competidor"].apply(retorna_ID_competidor)
df_sumulas_novas["ID Competição"] = df_sumulas_novas["Nome Competição"].apply(retorna_ID_competicao)

In [287]:
df_final_data_sumulas["ID Competidor"] = df_final_data_sumulas["Nome Competidor"].apply(retorna_ID_competidor)
df_final_data_sumulas["ID Competição"] = df_final_data_sumulas["Nome Competição"].apply(retorna_ID_competicao)

#### Tratando as Flotilhas

In [288]:
df_sumulas_novas["Flotilha"].value_counts()

Flotilha
GERAL         13970
MEDAL RACE      628
Name: count, dtype: int64

In [289]:
df_sumulas_novas[df_sumulas_novas.columns[6]].value_counts()

Flag Name
OVER      10141
Silver     1794
Gold       1728
Bronze      935
Name: count, dtype: int64

In [290]:
mask = df_sumulas_novas["Flag Name"].isin(["Gold", "Silver", "Bronze"])

mapeamento = {"Gold": "OURO", "Silver": "PRATA", "Bronze": "BRONZE"}

df_sumulas_novas.loc[mask, "Flotilha"] = df_sumulas_novas.loc[mask, "Flag Name"].map(mapeamento)

In [291]:
df_sumulas_novas["Flotilha"].value_counts()

Flotilha
GERAL         9623
PRATA         1794
OURO          1728
BRONZE         935
MEDAL RACE     518
Name: count, dtype: int64

#### Arrumando as Colunas

In [292]:
df_final_data_sumulas.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição'],
      dtype='object')

In [293]:
df_sumulas_novas.columns

Index(['Identificador', 'Nome Competição', 'Regata', 'Nome Competidor',
       'Pontuação Regata (Nha)', 'Classe Vela', 'Flag Name', 'Posição Geral',
       'Pontuação Regata', 'Descarte', 'Punição', 'Flotilha', 'Total', 'Net',
       'ID Competidor', 'ID Competição'],
      dtype='object')

In [294]:
df_sumulas_novas = df_sumulas_novas.rename(columns={'Total': 'Pontuação Total', "Net":"Nett"})
df_sumulas_novas.columns

Index(['Identificador', 'Nome Competição', 'Regata', 'Nome Competidor',
       'Pontuação Regata (Nha)', 'Classe Vela', 'Flag Name', 'Posição Geral',
       'Pontuação Regata', 'Descarte', 'Punição', 'Flotilha',
       'Pontuação Total', 'Nett', 'ID Competidor', 'ID Competição'],
      dtype='object')

In [295]:
colunas_extras = ["Identificador", "Regata", "Pontuação Regata (Nha)", "Flag Name"]
df_sumulas_novas = df_sumulas_novas.drop(columns=colunas_extras)
df_sumulas_novas.columns

Index(['Nome Competição', 'Nome Competidor', 'Classe Vela', 'Posição Geral',
       'Pontuação Regata', 'Descarte', 'Punição', 'Flotilha',
       'Pontuação Total', 'Nett', 'ID Competidor', 'ID Competição'],
      dtype='object')

In [296]:
ordem_colunas = ['ID Competidor', 'Nome Competidor', 'ID Competição',
                 'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
                 'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
                 'Nome Competição']

df_sumulas_novas = df_sumulas_novas.reindex(columns=ordem_colunas)
df_sumulas_novas.columns

Index(['ID Competidor', 'Nome Competidor', 'ID Competição', 'Classe Vela',
       'Pontuação Regata', 'Descarte', 'Flotilha', 'Posição Geral', 'Punição',
       'Pontuação Total', 'Nett', 'Nome Competição'],
      dtype='object')

In [297]:
df_final_data_sumulas = df_final_data_sumulas.drop(columns=df_final_data_sumulas.columns[0])
df_final_data_sumulas.columns

Index(['ID Competidor', 'Nome Competidor', 'ID Competição', 'Classe Vela',
       'Pontuação Regata', 'Descarte', 'Flotilha', 'Posição Geral', 'Punição',
       'Pontuação Total', 'Nett', 'Nome Competição'],
      dtype='object')

#### Merge Dos Dados

In [298]:
df_data_final_atualizado = pd.concat([df_final_data_sumulas, df_sumulas_novas]).reset_index(drop=True)

In [299]:
df_data_final_atualizado["ID Resultado"] = df_data_final_atualizado.index + 1

In [300]:
ordem_colunas = ['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
                 'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
                 'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
                 'Nome Competição']

df_data_final_atualizado = df_data_final_atualizado.reindex(columns=ordem_colunas)
df_data_final_atualizado.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição'],
      dtype='object')

In [301]:
df_data_final_atualizado.head()

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição
0,1,1,"WILLIAM JONES, EVAN DEPAUL",17,49er,11,1,GERAL,1,UFD,44.0,33.0,World Championship 2017
1,2,2,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7,49er,3,0,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022
2,3,2,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7,49er,1,0,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022
3,4,2,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7,49er,1,0,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022
4,5,2,"DOMINIK BUKSAK, SZYMON WIERZBICKI",7,49er,3,0,GERAL,1,NaN,89.0,66.0,Semaine Olympique Francaise De Voile 2022


In [302]:
df_data_final_atualizado.tail()

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição
151211,151212,715,"ZOFIA BURSKA, SARA TKADLECOVA",34,49erfx,27.0,0,GERAL,47,DNC,334.0,307.0,World Championship 2024
151212,151213,3370,"PAULA VAN WIERINGEN MASSANET, CLAUDIA SOBRAL L...",34,49erfx,22.0,0,GERAL,48,0,342.0,315.0,World Championship 2024
151213,151214,3321,"MAELLE FRASCARI, SVEVA CARRARO",34,49erfx,18.0,0,GERAL,49,0,347.0,320.0,World Championship 2024
151214,151215,3313,"MANASE ICHIHASHI, RINKO GOTO",34,49erfx,21.0,0,GERAL,50,0,352.0,325.0,World Championship 2024
151215,151216,3494,"JOSEFINE NOJGAARD , ESTHER BOJSEN-MOLLER",34,49erfx,27.0,0,GERAL,51,DNC,355.0,328.0,World Championship 2024


#### Ajustes Nos Dados

In [303]:
df_data_final_atualizado["Classe Vela"].value_counts()

Classe Vela
Ilca 7                34279
49er                  25746
Ilca 6                24660
49erFX                16071
IQFOIL Masc.          11186
IQFOIL Fem.            8384
Formula Kite Masc.     6014
470.0                  4540
Nacra 17               3661
Formula Kite Fem.      3259
ILCA 7                 2626
IQFoil Masc.           2596
IQFoil Fem.            2162
ILCA 6                 1881
Kite Masc.             1720
Kite Fem.               969
49erfx                  791
470                     671
Name: count, dtype: int64

In [304]:
def ajusta_classe(valor):
    if valor.upper() == "KITE MASC.":
        return "FORMULA KITE MASC."
    elif valor.upper() == "KITE FEM.":
        return "FORMULA KITE FEM."
    elif str(valor) == "470.0":
        return "470"
    return str(valor).upper()

In [305]:
df_data_final_atualizado["Classe Vela"] = df_data_final_atualizado["Classe Vela"].apply(ajusta_classe)

In [306]:
df_data_final_atualizado["Classe Vela"].value_counts()

Classe Vela
ILCA 7                36905
ILCA 6                26541
49ER                  25746
49ERFX                16862
IQFOIL MASC.          13782
IQFOIL FEM.           10546
FORMULA KITE MASC.     7734
470                    5211
FORMULA KITE FEM.      4228
NACRA 17               3661
Name: count, dtype: int64

In [307]:
df_data_final_atualizado["Flotilha"].value_counts()

Flotilha
GERAL         80073
OURO          31064
PRATA         27859
BRONZE        10040
MEDAL RACE     2180
Name: count, dtype: int64

In [308]:
df_data_final_atualizado["Punição"].value_counts()

Punição
0      13627
DNC     4793
BFD     1803
UFD     1106
DNF      881
DSQ      368
RET      342
STP      322
RDG      241
DNS      123
DNE       76
DPI       54
OCS       31
SCP       29
SPI       26
DP        11
SP1       10
10%        7
ARB        7
PRP        6
30%        6
NSC        3
SP5        2
RAF        2
DCT        1
Name: count, dtype: int64

#### Salvando os Dados

In [309]:
df_final_data_competicoes.to_excel("dados_finais_2024/competicoes.xlsx", index=False)
df_final_data_atletas.to_excel("dados_finais_2024/atletas.xlsx", index=False)
df_data_final_atualizado.to_excel("dados_finais_2024/sumulas.xlsx", index=False)

In [312]:
with pd.ExcelWriter('dados_finais_2024/final_data.xlsx') as writer:
    # Salvando cada DataFrame em uma planilha diferente
    df_data_final_atualizado.to_excel(writer, sheet_name='Súmulas', index=False)
    df_final_data_atletas.to_excel(writer, sheet_name='Atletas', index=False)
    df_final_data_competicoes.to_excel(writer, sheet_name='Competições', index=False)